# Защита персональных данных клиентов

## Цель

Защитить данные клиентов страховой компании «Хоть потоп». 

## Задачи

1. Разработаем метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 
1. Обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка необходимых библиотек и набора данных

In [1]:
import warnings

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
pd.set_option("display.precision", 2)
pd.set_option("display.max_columns", 100)
pd.options.mode.chained_assignment = None

warnings.filterwarnings("ignore")

In [3]:
try:
    df = pd.read_csv("./insurance.csv")
except FileNotFoundError:
    df = pd.read_csv("/datasets/insurance.csv")

In [4]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.0,5000.00,5000.00,5000.00,5000.00
mean,0.5,30.95,39916.36,1.19,0.15
std,0.5,8.44,9900.08,1.09,0.46
min,0.0,18.00,5300.00,0.00,0.00
25%,0.0,24.00,33300.00,0.00,0.00
50%,0.0,30.00,40200.00,1.00,0.00
75%,1.0,37.00,46600.00,2.00,0.00
max,1.0,65.00,79000.00,6.00,5.00


In [7]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Данные загрузились корректно, типы полей корректны, явные пропуски отсутствуют.

## Обфускация данных клиентов

Наша модель линейной регрессии выглядит как
$$
y = Xw 
$$

Обучение модели сводится к минимизации невязки прогноза и истинных значений:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула линейной регрессии:

$$
w = (X^T X)^{-1} X^T y \tag{1}
$$

Предсказания модели можно записать как 
$$
a = Xw \tag{2} 
$$

где
- $X$ — матрица признаков (нулевой столбец состоит из единиц)
- $y$ — вектор целевого признака
- $a$ — вектор прогнозов
- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

В качестве варианта защиты данных можно умножить нашу матрицу признаком $X$ (те самые персональные данные) на обратимую матрицу $P$. 

В таком случае мы получим линейную регрессию вида 
\begin{equation*}
XPu=Py \tag{3}
\end{equation*} что допустимо, в силу обратимости матрицы $P$. Для пущего единообразия можем переобозначить величины: $$XP=Z$$ так что получим 
$$
Zu=a' \tag{4}
$$

Остаётся убедиться, что для новой матрицы признаков из $(4)$ выполняется условие $(1)$ 

Для $u$ из (4)

$$ u = (Z^T Z)^{-1} Z^T y\tag{5}$$

Подставив (5) в (4) получим

$$a'=Z(Z^T Z)^{-1} Z^T y$$

После подставновки значения $Z$ получаем

$$
a' = XP ((XP)^T (XP))^{-1} (XP)^T y
$$

Воспользуемся тем, что 

$$(AB)^{T} = B^TA^T$$
    $$(AB)^{-1} = B^{-1}A^{-1}$$
    $$ AA^{-1} = E $$
    
Заметим, что 

$$
((XP)^T XP)^{-1} = (P^T (X^T X) P)^{-1} = P^{-1} (X^T X)^{-1} (P^T)^{-1}
$$

Таким образом
$$a'=XP P^{-1} (X^T X)^{-1} (P^T)^{-1} (XP)^T y = X \left[P P^{-1}\right] (X^T X)^{-1}\left[(P^T)^{-1}P^T\right] X^Ty$$

Заметим, что $PP^{-1}=E$ и $\left(P^T\right)^{-1}P^T=E$

Но это означает, что

$$
a' = X\left( X^TX \right)^{-1} X^T y = Xw = a
$$

Таким образом, мы получаем, что прогноз модели, обфусциорванной умножеением на обратимую матрицу совпадает с прогнозом исходной модели.

Таким образом, мы показали, умножение матрицы признаков, содержащей персональные данные клиентов, приводит к "сокрытию" этих данных (делает неочевидной их интерпретацию), но, при этом, не ломает регрессию.  

## Вычисление линейной регрессии на исходных и обфусцированных данных 

Первым делом обучим линейную регрессию на исходных данных

In [8]:
X = df.drop(["Страховые выплаты"], axis=1)
y = df["Страховые выплаты"]

Поскольку у нас нет тестовой выборки, используем для этого часть исходных данных.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

In [10]:
regression = LinearRegression().fit(X_train, y_train)

In [11]:
display(
    f"R2 score on clear personal date: {r2_score(y_test, regression.predict(X_test)):0.4f}"
)

'R2 score on clear personal date: 0.4255'

### Обфускация

Теперь нам нужно сгенерировать обратимую матрицу, которую мы используем для "шифрования" данных. Исходная матрица $X$ содержит 4 признака, значит нам нужно построить квадратную матрицу $[4\times 4]$


Характеристическим признаком обратимой матрицы является ненулевой определитель, воспользуемся этим фактом. 

In [12]:
while True:
    M = np.random.random((4, 4))
    if np.linalg.det(M) != 0:
        break

In [13]:
M

array([[0.87654886, 0.04632416, 0.16778134, 0.65677565],
       [0.11955524, 0.70970385, 0.5195768 , 0.55386679],
       [0.03619994, 0.19293082, 0.78169508, 0.06227338],
       [0.42766442, 0.4393048 , 0.49531451, 0.01359199]])

In [14]:
np.linalg.det(M)

-0.22288792157972032

Построим матрицу обфусцированных данных и обучим на ней линейную регрессию

In [15]:
Xob_train = np.dot(X_train, M)
Xob_test = np.dot(X_test, M)

In [16]:
Xob_train.shape, X_train.shape

((3750, 4), (3750, 4))

In [17]:
Xob_test.shape, X_test.shape

((1250, 4), (1250, 4))

In [18]:
pd.DataFrame(data=Xob_train, columns=X_train.columns).head()

,Пол,Возраст,Зарплата,Члены семьи
0,1459.30,7779.95,31441.81,2522.22
1,1208.64,6441.36,26042.89,2086.46
2,1582.38,8435.91,34099.57,2733.95
3,1258.13,6701.44,27066.02,2175.27
4,1488.93,7936.15,32068.70,2573.16


Уже по первым пяти строкам данных видно, что интерпретировать значения из новой таблицы будет не просто.

In [19]:
regression_ob = LinearRegression().fit(Xob_train, y_train)

In [20]:
display(
    f"R2 score on clear personal date: {r2_score(y_test, regression.predict(X_test)):0.4f}"
)
display(
    f"R2 score on obfuscated personal date: {r2_score(y_test, regression_ob.predict(Xob_test)):0.4f}"
)
display(
    f"R2 score difference: {r2_score(y_test, regression.predict(X_test))-r2_score(y_test, regression_ob.predict(Xob_test))}"
)

'R2 score on clear personal date: 0.4255'

'R2 score on obfuscated personal date: 0.4255'

'R2 score difference: 1.2212453270876722e-15'

Как видим, точность регрессии в процессе обфускации не пострадала: разница между мерами R2 в 13м знаке после запятой соответсвует погрешности вычислений. 

# Вывод

Мы смогли создать простой и действенный способ защиты персональных данных. Используя это простой метод мы можем без риска утечки ПД передавать массивы данных о клиентах и проводить необходимые вычисления (по крайней мере на линейных моделях), в том числе на сторонних вычислительных ресурсах. 